In [1]:
import pandas as pd
import os
import cv2
import numpy as np
from tqdm.notebook import tqdm
from ultralytics import YOLO
import shutil
from sklearn.model_selection import train_test_split

In [2]:
# Laad de handmatige annotaties
annotations = pd.read_csv('Data/input/annotaties_handmatig.csv')

def load_annotated_images(annotations):
    images = []
    labels = []
    for index, row in annotations.iterrows():
        image_path = os.path.join('Data/input/images_resized_annotated', row['image_name'])
        image = cv2.imread(image_path)
        bbox = [row['bbox_x'], row['bbox_y'], row['bbox_x'] + row['bbox_width'], row['bbox_y'] + row['bbox_height']]
        images.append(image)
        labels.append(bbox)
    return images, labels

annotated_images, labels = load_annotated_images(annotations)

In [3]:
def prepare_yolo_data(annotations, images_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for index, row in annotations.iterrows():
        image_path = os.path.join(images_folder, row['image_name'])
        image = cv2.imread(image_path)
        h, w, _ = image.shape
        x_center = (row['bbox_x'] + row['bbox_width'] / 2) / w
        y_center = (row['bbox_y'] + row['bbox_height'] / 2) / h
        width = row['bbox_width'] / w
        height = row['bbox_height'] / h
        label_path = os.path.join(output_folder, f"{os.path.splitext(row['image_name'])[0]}.txt")
        with open(label_path, 'w') as f:
            f.write(f"0 {x_center} {y_center} {width} {height}\n")

prepare_yolo_data(annotations, 'Data/input/images_resized_annotated', 'D4_AL_Yolo-V8_Data/yolo_labels')


In [4]:
# Maak directories voor training en validatie afbeeldingen en labels
os.makedirs('D4_AL_Yolo-V8_Data/train/images', exist_ok=True)
os.makedirs('D4_AL_Yolo-V8_Data/train/labels', exist_ok=True)
os.makedirs('D4_AL_Yolo-V8_Data/val/images', exist_ok=True)
os.makedirs('D4_AL_Yolo-V8_Data/val/labels', exist_ok=True)

# Krijg een lijst van alle afbeeldingen
images = [f for f in os.listdir('Data/input/images_resized_annotated') if f.endswith('.jpg')]

# Splits de data
train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)

# Kopieer de afbeeldingen en annotaties naar de juiste mappen
for img in train_images:
    shutil.copy(os.path.join('Data/input/images_resized_annotated', img), 'D4_AL_Yolo-V8_Data/train/images')
    shutil.copy(os.path.join('D4_AL_Yolo-V8_Data/yolo_labels', os.path.splitext(img)[0] + '.txt'), 'D4_AL_Yolo-V8_Data/train/labels')

for img in val_images:
    shutil.copy(os.path.join('Data/input/images_resized_annotated', img), 'D4_AL_Yolo-V8_Data/val/images')
    shutil.copy(os.path.join('D4_AL_Yolo-V8_Data/yolo_labels', os.path.splitext(img)[0] + '.txt'), 'D4_AL_Yolo-V8_Data/val/labels')


In [5]:
dataset_yaml_content = """
train: C:/Users/Gebruiker/Documents/Fontys/S6 - AI/Personal - Data Driven Challenge/arise-challenge-algorithm/D4_AL_Yolo-V8_Data/train/images
val: C:/Users/Gebruiker/Documents/Fontys/S6 - AI/Personal - Data Driven Challenge/arise-challenge-algorithm/D4_AL_Yolo-V8_Data/val/images

nc: 1
names: ['Insect']
"""

with open('D4_AL_Yolo-V8_Data/dataset-m.yaml', 'w') as f:
    f.write(dataset_yaml_content)

In [6]:
# Laad het YOLOv8 nano model
model = YOLO('yolov8m.yaml')  # Dit kan ook een andere YOLOv8 configuratie zijn

# Train het model met early stopping
model.train(data='D4_AL_Yolo-V8_Data/dataset-m.yaml', epochs=10, imgsz=128)


Ultralytics YOLOv8.2.30  Python-3.11.1 torch-2.2.2+cpu CPU (AMD Ryzen 7 5700U with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8m.yaml, data=D4_AL_Yolo-V8_Data/dataset-m.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=128, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True

train: Scanning C:\Users\Gebruiker\Documents\Fontys\S6 - AI\Personal - Data Driven Challenge\arise-challenge-algorithm\D4_AL_Yolo-V8_Data\train\labels.cache... 40 images, 0 backgrounds, 0 corrupt: 100%|██████████| 40/40 [00:00<?, ?it/s]
val: Scanning C:\Users\Gebruiker\Documents\Fontys\S6 - AI\Personal - Data Driven Challenge\arise-challenge-algorithm\D4_AL_Yolo-V8_Data\val\labels.cache... 10 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10/10 [00:00<?, ?it/s]


Plotting labels to runs\detect\train5\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 128 train, 128 val
Using 0 dataloader workers
Logging results to runs\detect\train5
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10         0G      3.599      3.747      4.225          8        128: 100%|██████████| 3/3 [00:08<00:00,  2.93s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.55s/it]

                   all         10         10     0.0267        0.6       0.18     0.0428


...

                   all         10         10     0.0261        0.6      0.185     0.0508



10 epochs completed in 0.036 hours.
Optimizer stripped from runs\detect\train5\weights\last.pt, 52.0MB
Optimizer stripped from runs\detect\train5\weights\best.pt, 52.0MB

Validating runs\detect\train5\weights\best.pt...
Ultralytics YOLOv8.2.30  Python-3.11.1 torch-2.2.2+cpu CPU (AMD Ryzen 7 5700U with Radeon Graphics)
YOLOv8m summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.09s/it]


                   all         10         10     0.0261        0.6      0.186     0.0566
Speed: 0.2ms preprocess, 106.0ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs\detect\train5


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000169B9DF48D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [7]:
def calculate_uncertainty(predictions):
    confidences = [min([box.conf[0] for box in result.boxes]) if result.boxes else 0 for result in predictions]
    return confidences

In [8]:
def predict_and_select_uncertain(model, image_folder, num_samples=10, batch_size=50):
    image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder)]
    all_confidences = []
    all_image_paths = []
    
    for i in tqdm(range(0, len(image_paths), batch_size), desc="Predicting batches"):
        batch_paths = image_paths[i:i+batch_size]
        results = model.predict(batch_paths, verbose=False)
        
        confidences = calculate_uncertainty(results)
        all_confidences.extend(confidences)
        all_image_paths.extend(batch_paths)
    
    uncertain_indices = np.argsort(all_confidences)[:num_samples]
    uncertain_images = [all_image_paths[i] for i in uncertain_indices]
    
    return uncertain_images

# Selecteer de meest onzekere afbeeldingen
uncertain_images = predict_and_select_uncertain(model, 'Data/input/images_resized')

Predicting batches:   0%|          | 0/789 [00:00<?, ?it/s]

In [9]:
# Stap 8: Resultaten Bekijken
print("Meest onzekere afbeeldingen:", uncertain_images)

Meest onzekere afbeeldingen: ['Data/input/images_resized\\102_20200628004052_18636.jpg', 'Data/input/images_resized\\192_20200724112811_1860_t.jpg', 'Data/input/images_resized\\192_20200724112811_1861_t.jpg', 'Data/input/images_resized\\192_20200724112811_1862_t.jpg', 'Data/input/images_resized\\192_20200724112811_1863_t.jpg', 'Data/input/images_resized\\192_20200724112811_1864_t.jpg', 'Data/input/images_resized\\192_20200724112811_1865_t.jpg', 'Data/input/images_resized\\192_20200724112811_1866_t.jpg', 'Data/input/images_resized\\192_20200724112811_1867_t.jpg', 'Data/input/images_resized\\192_20200724112811_1868_t.jpg']


In [10]:
# Model Opslaan
model.export(format='torchscript')  # Opslaan als TorchScript-bestand

Ultralytics YOLOv8.2.30  Python-3.11.1 torch-2.2.2+cpu CPU (AMD Ryzen 7 5700U with Radeon Graphics)

PyTorch: starting from 'runs\detect\train5\weights\best.pt' with input shape (1, 3, 128, 128) BCHW and output shape(s) (1, 5, 336) (49.6 MB)

TorchScript: starting export with torch 2.2.2+cpu...
TorchScript: export success  2.3s, saved as 'runs\detect\train5\weights\best.torchscript' (99.0 MB)

Export complete (4.3s)
Results saved to C:\Users\Gebruiker\Documents\Fontys\S6 - AI\Personal - Data Driven Challenge\arise-challenge-algorithm\runs\detect\train5\weights
Predict:         yolo predict task=detect model=runs\detect\train5\weights\best.torchscript imgsz=128  
Validate:        yolo val task=detect model=runs\detect\train5\weights\best.torchscript imgsz=128 data=D4_AL_Yolo-V8_Data/dataset-m.yaml  
Visualize:       https://netron.app


'runs\\detect\\train5\\weights\\best.torchscript'